# CA Watersheds: Fire Density 1992-2020

In [3]:
import os
import warnings

import earthpy as et
import geopandas as gpd
import geoviews as gv
import holoviews as hv
import hvplot.pandas
import pandas as pd
import pyogrio


# Ignore FutureWarning coming from hvplot
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [4]:
wbd_national_gdb_url = ('https://prd-tnm.s3.amazonaws.com'
                        '/StagedProducts/Hydrography/WBD'
                        '/National/GDB/WBD_National_GDB.zip')
wbd_dir = et.data.get_data(url=wbd_national_gdb_url)
# wbd_dir

In [5]:
wbd_path = os.path.join(wbd_dir, 'WBD_National_GDB.gdb')
# wbd_path

In [6]:
# import fiona

# fiona.listlayers(f'{wbd_path}')

In [7]:
wbd_path = os.path.join(wbd_dir, 'WBD_National_GDB.gdb')
wbd_gdf = gpd.read_file(wbd_path, driver='OpenFileGBD', layer='WBDHU4', from_disk=True)

In [8]:
wbd_gdf['states'] = wbd_gdf['states'].fillna('NULL')

In [9]:
ca_wbd_gdf = wbd_gdf[wbd_gdf['states'].str.contains('CA')]
# ca_wbd_gdf

In [10]:
usa_fire_url = (
    'https://www.fs.usda.gov'
    '/rds/archive/products/RDS-2013-0009.6'
    '/RDS-2013-0009.6_Data_Format2_GDB.zip'
)
fire_dir = et.data.get_data(url=usa_fire_url)
# fire_dir

In [11]:
fire_path = os.path.join(fire_dir, 'Data', 'FPA_FOD_20221014.gdb')
if not 'fire_gdf' in globals():
    fire_gdf = pyogrio.read_dataframe(fire_path, layer='Fires')
# fire_gdf.head()

In [12]:
fire_gdf['STATE'] = fire_gdf['STATE'].fillna('NULL')
fire_gdf = fire_gdf[fire_gdf['STATE'].str.contains('CA')]
# fire_gdf

In [13]:
fire_clean_gdf = (
    fire_gdf
    [['FOD_ID', 'DISCOVERY_DATE', 'FIRE_SIZE', 'geometry']]
    .set_index('FOD_ID')
)
fire_clean_gdf.DISCOVERY_DATE = pd.to_datetime(fire_clean_gdf.DISCOVERY_DATE)
# print(fire_clean_gdf.crs)
fire_clean_gdf = fire_clean_gdf.to_crs(wbd_gdf.crs)
# print(fire_clean_gdf.crs)
# fire_clean_gdf #.info()

In [14]:
fire_region_gdf = (
    wbd_gdf
    [['name', 'geometry']]
    .sjoin(fire_clean_gdf, how='inner', predicate='intersects')
)
fire_region_gdf = (
    fire_region_gdf
    .groupby(['name', fire_region_gdf.DISCOVERY_DATE.dt.year])
    .agg(
        max_fire_size=('FIRE_SIZE', 'max'), 
        num_fires=('index_right', 'count'))
)
# fire_region_gdf.num_fires.sum()
# fire_region_gdf

In [15]:
# fire_region_gdf.reset_index()['DISCOVERY_DATE'].sort_values().unique()

In [16]:
# Compute the area of regional watersheds
ca_wbd_gdf['area_ha'] = (
                    ca_wbd_gdf.to_crs(epsg=9822).area 
                    # Convert to hectares
                    / 10000
                    # Convert to millions of hectares
                    / 1000000
)

# Compute total fires in each watershed
fire_count_df = (
    fire_region_gdf
    .reset_index()
    [['name', 'num_fires']]
    .groupby('name')
    .sum())

fire_density_gdf = (
    ca_wbd_gdf
    .set_index('name')
    # Add the area and geometry back in
    .join(fire_count_df)
    [['num_fires', 'area_ha', 'geometry']]
)

# Compure fire density
fire_density_gdf['fire_density_per_million_ha'] = (
    fire_density_gdf.num_fires / fire_density_gdf.area_ha
)

fire_density_gdf[['fire_density_per_million_ha']].sort_values(by='fire_density_per_million_ha', ascending=False)

,fire_density_per_million_ha
name,
Southern California Coastal,12965.741336
San Joaquin,10325.877112
San Francisco Bay,8108.829374
Sacramento,7497.497194
Tulare-Buena Vista Lakes,5854.976782
Klamath-Northern California Coastal,4212.210558
Central California Coastal,4011.792797
North Lahontan,2471.083569
Southern Mojave-Salton Sea,2171.795968


## CA Watersheds: Fire Density Chloropleth Plot

In [17]:
fire_density_gdf.geometry = fire_density_gdf.geometry.simplify(tolerance=.1)

In [23]:
poly_plot = (
    gv.Polygons(
        fire_density_gdf
            .reset_index()
            .dropna()
            [['fire_density_per_million_ha', 'name', 'geometry']])
    .opts(
        width=1000, height=600,
        colorbar=True, color='fire_density',
        cmap='plasma', line_color='white',
        xaxis='bare', yaxis='bare', tools=['hover']
    )
)

# import cartopy.crs as ccrs
# # (gv.tile_sources.OSM * poly_plot).opts(
# #     projection=ccrs.PlateCarree(central_longitude=-121))
# (poly_plot).opts(
#     data_aspect=1,
#     projection=ccrs.PlateCarree(central_longitude=-121))

gv.tile_sources.StamenTerrain * poly_plot

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]   (fire_density_per_million_ha,name)

# Fire Occurrence and Size Over Time for CA Watersheds

In [24]:
# Set values for ylabels and titles
labels = pd.DataFrame(dict(
    column_name = ['max_fire_size', 'num_fires'],
    ylabel = ['Fire Size (million ha)', 'Number of Fires'],
    title = ['Largest Fire on record in the region',
              'Total number of fires in the region']))
# print(labels)
def fire_plot(region_name, df=fire_region_gdf, labels=labels):
    """
    Create a multi-panel plot for a region

    Parameters
    ----------
    region_name : str
      The name of the region to generate a plot for. Must exists 
      in the 'name' index of df.
    df : pd.DataFrame
      The dataframe with the data to plot. Columns much match
      an item in labels.column_name to be plotted
    labels : pd.DataFrame
      Plot labels. Must have a 'column_name', 'ylabel', and 'title'
      columns with str values. Each row will be a subplot.

    Returns
    -------
    plot : hv.core.layout.Layout
      A holoviews plot layout or similar. For use with hv.DynamicMap.
    """
    # Generate a subplot for each row in the labels
    subplots = []
    # Iterate through the labels row by row
    for i, labs in labels.iterrows():
        # Create subplot
        subplot = (
            df.xs(region_name, level='name')
            [[labs.column_name]]
            .hvplot(
                xlabel='Year', ylabel=labs.ylabel, title=labs.title,
                width=800, color='red', size=2
            ))
        subplots.append(subplot)

    # Stack the subplots vertically
    plot = hv.Layout(subplots).cols(1)
    return plot

# Create a dropdown menu to switch between regions
(
    hv.DynamicMap(
        # The plotting function for the two-panel fire history
        fire_plot,
        # Define the dimension for the dropdown
        kdims=[('region', 'Region')])
    # Add the explicit indexing - region names as a bokeh dimension
    .redim.values(region=fire_region_gdf.reset_index().name)
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'2e1abea4-9480-443a-8e9e-1c3598f4222c': {'version…

# Citation
U.S. Geological Survey, National Geospatial Technical Operations Center, 2023, Watershed Boundary Dataset (WBD) - USGS National Map Downloadable Data Collection: U.S. Geological Survey.

Short, Karen C. 2022. Spatial wildfire occurrence data for the United States, 1992-2020 [FPA_FOD_20221014]. 6th Edition. Fort Collins, CO: Forest Service Research Data Archive. https://doi.org/10.2737/RDS-2013-0009.6

In [20]:
%%capture
%%bash
jupyter nbconvert 02-personal-wildfire.ipynb --to html --no-input